In [ ]:

# Tesla MILP Model Visualization in Jupyter Notebook

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyomo.environ import *
from pyomo.opt import SolverFactory

# 1. Define the model
model = ConcreteModel()
model.S = Set(initialize=['Chile_Lithium', 'Indonesia_Nickel'])
model.P = Set(initialize=['Giga_Nevada', 'Giga_Berlin', 'Giga_Texas'])
model.C = Set(initialize=['Fremont', 'Shanghai'])
model.T = Set(initialize=[1, 2, 3])

model.demand = Param(model.C, initialize={'Fremont': 90000, 'Shanghai': 110000})
model.capacity = Param(model.P, initialize={'Giga_Nevada': 100000, 'Giga_Berlin': 80000, 'Giga_Texas': 120000})
model.fixed_cost = Param(model.P, initialize={'Giga_Nevada': 1000000, 'Giga_Berlin': 950000, 'Giga_Texas': 1100000})
model.energy_usage = Param(model.P, initialize={'Giga_Nevada': 150, 'Giga_Berlin': 140, 'Giga_Texas': 160})
model.energy_cap = Param(model.P, initialize={'Giga_Nevada': 1.5e7, 'Giga_Berlin': 1.2e7, 'Giga_Texas': 1.8e7})
model.energy_price = Param(model.T, initialize={1: 0.08, 2: 0.10, 3: 0.12})
model.trans_sp = Param(model.S, model.P, initialize={
    ('Chile_Lithium', 'Giga_Nevada'): 5, ('Chile_Lithium', 'Giga_Berlin'): 12, ('Chile_Lithium', 'Giga_Texas'): 6,
    ('Indonesia_Nickel', 'Giga_Nevada'): 7, ('Indonesia_Nickel', 'Giga_Berlin'): 4, ('Indonesia_Nickel', 'Giga_Texas'): 5
})
model.trans_pc = Param(model.P, model.C, initialize={
    ('Giga_Nevada', 'Fremont'): 3, ('Giga_Nevada', 'Shanghai'): 8,
    ('Giga_Berlin', 'Fremont'): 10, ('Giga_Berlin', 'Shanghai'): 6,
    ('Giga_Texas', 'Fremont'): 4, ('Giga_Texas', 'Shanghai'): 7
})

model.x = Var(model.S, model.P, domain=NonNegativeReals)
model.y = Var(model.P, model.C, domain=NonNegativeReals)
model.open_p = Var(model.P, domain=Binary)
model.energy_used = Var(model.P, model.T, domain=NonNegativeReals)

model.obj = Objective(expr=
    sum(model.trans_sp[s, p] * model.x[s, p] for s in model.S for p in model.P) +
    sum(model.trans_pc[p, c] * model.y[p, c] for p in model.P for c in model.C) +
    sum(model.fixed_cost[p] * model.open_p[p] for p in model.P) +
    sum(model.energy_used[p, t] * model.energy_price[t] for p in model.P for t in model.T),
    sense=minimize)

model.demand_con = Constraint(model.C, rule=lambda m, c: sum(m.y[p, c] for p in m.P) >= m.demand[c])
model.capacity_con = Constraint(model.P, rule=lambda m, p: sum(m.y[p, c] for c in m.C) <= m.capacity[p] * m.open_p[p])
model.balance_con = Constraint(model.P, rule=lambda m, p: sum(m.x[s, p] for s in m.S) == sum(m.y[p, c] for c in m.C))
model.energy_cap_con = Constraint(model.P, rule=lambda m, p: sum(m.energy_used[p, t] for t in m.T) <= m.energy_cap[p])
model.energy_use_con = Constraint(model.P, rule=lambda m, p: sum(m.energy_used[p, t] for t in m.T) >= m.energy_usage[p] * sum(m.y[p, c] for c in m.C))

solver = SolverFactory('glpk')
solver.solve(model)

# 2. Collect Results
df_energy = pd.DataFrame([
    {"Plant": p, "Time": t, "Energy_kWh": model.energy_used[p, t]()} 
    for p in model.P for t in model.T if model.energy_used[p, t]() > 0
])
df_product = pd.DataFrame([
    {"Plant": p, "Customer": c, "Units": model.y[p, c]()} 
    for p in model.P for c in model.C if model.y[p, c]() > 0
])
df_status = pd.DataFrame({
    "Plant": list(model.P),
    "Status": ['Open' if model.open_p[p]() > 0.5 else 'Closed' for p in model.P]
})

# 3. Visualizations
sns.set(style="whitegrid")

# Plant Status
plt.figure(figsize=(6, 3))
sns.barplot(data=df_status, x="Plant", y="Status")
plt.title("Plant Operational Status")
plt.show()

# Product Flow
plt.figure(figsize=(8, 4))
sns.barplot(data=df_product, x="Plant", y="Units", hue="Customer")
plt.title("Battery Flow from Plant to Customer")
plt.ylabel("Units Produced")
plt.show()

# Energy Use
plt.figure(figsize=(8, 4))
sns.barplot(data=df_energy, x="Plant", y="Energy_kWh", hue="Time")
plt.title("Energy Use by Time Slot")
plt.ylabel("Energy (kWh)")
plt.show()
